In [1]:
from langchain.tools import tool

## Langchain Tools Library
Langchain has built integration with a lot of different tools. You can see their current list of integrations below.

https://python.langchain.com/docs/integrations/tools

### Arxiv Tool

In [2]:
from langchain_community.tools.arxiv.tool import ArxivQueryRun, ArxivAPIWrapper
arxiv_tool = ArxivQueryRun(api_wrapper=ArxivAPIWrapper())
arxiv_tool.name, arxiv_tool.description


('arxiv',
 'A wrapper around Arxiv.org Useful for when you need to answer questions about Physics, Mathematics, Computer Science, Quantitative Biology, Quantitative Finance, Statistics, Electrical Engineering, and Economics from scientific articles on arxiv.org. Input should be a search query.')

In [3]:
arxiv_tool.run("Tell me about Llama 2: Open Foundation and Fine-Tuned Chat Models")

'Published: 2024-04-11\nTitle: Me LLaMA: Foundation Large Language Models for Medical Applications\nAuthors: Qianqian Xie, Qingyu Chen, Aokun Chen, Cheng Peng, Yan Hu, Fongci Lin, Xueqing Peng, Jimin Huang, Jeffrey Zhang, Vipina Keloth, Xinyu Zhou, Huan He, Lucila Ohno-Machado, Yonghui Wu, Hua Xu, Jiang Bian\nSummary: Recent advancements in large language models (LLMs) such as ChatGPT and LLaMA\nhave hinted at their potential to revolutionize medical applications, yet their\napplication in clinical settings often reveals limitations due to a lack of\nspecialized training on medical-specific data. In response to this challenge,\nthis study introduces Me-LLaMA, a novel medical LLM family that includes\nfoundation models - Me-LLaMA 13/70B, along with their chat-enhanced versions -\nMe-LLaMA 13/70B-chat, developed through continual pre-training and instruction\ntuning of LLaMA2 using large medical datasets. Our methodology leverages a\ncomprehensive domain-specific data suite, including a 

### Wikidata Tool

In [4]:
from langchain_community.tools.wikidata.tool import WikidataAPIWrapper, WikidataQueryRun

wikidata_tool = WikidataQueryRun(api_wrapper=WikidataAPIWrapper())
wikidata_tool.name, wikidata_tool.description

('Wikidata',
 'A wrapper around Wikidata. Useful for when you need to answer general questions about people, places, companies, facts, historical events, or other subjects. Input should be the exact name of the item you want information about or a Wikidata QID.')

In [5]:
print(wikidata_tool.run("Alan Turing"))

Result Q7251:
Label: Alan Turing
Description: English computer scientist (1912–1954)
Aliases: Alan M. Turing, Alan Mathieson Turing, Turing, Alan Mathison Turing
instance of: human
country of citizenship: United Kingdom
occupation: computer scientist, mathematician, university teacher, cryptographer, logician, statistician, marathon runner, artificial intelligence researcher
sex or gender: male
date of birth: 1912-06-23
date of death: 1954-06-07
sport: athletics
place of birth: Maida Vale, Warrington Lodge
educated at: King's College, Princeton University, Sherborne School, Hazlehurst Community Primary School
employer: Victoria University of Manchester, Government Communications Headquarters, University of Cambridge, National Physical Laboratory (United Kingdom)
place of death: Wilmslow
field of work: cryptanalysis, computer science, mathematics, logic, cryptography
cause of death: cyanide poisoning
notable work: On Computable Numbers, with an Application to the Entscheidungsproblem, C

### Youtube Search 

In [6]:
from langchain_community.tools import YouTubeSearchTool
youtube_tool = YouTubeSearchTool()
youtube_tool.name, youtube_tool.description

('youtube_search',
 'search for youtube videos associated with a person. the input to this tool should be a comma separated list, the first part contains a person name and the second a number that is the maximum number of video results to return aka num_results. the second part is optional')

In [7]:
youtube_tool.run("stephen colbert,5")

"['https://www.youtube.com/watch?v=h6Udx4UOGvA&pp=ygUPc3RlcGhlbiBjb2xiZXJ0', 'https://www.youtube.com/watch?v=HQIqWLu0yA8&pp=ygUPc3RlcGhlbiBjb2xiZXJ0', 'https://www.youtube.com/watch?v=aM3CVhgnw68&pp=ygUPc3RlcGhlbiBjb2xiZXJ0', 'https://www.youtube.com/watch?v=HeJH28Zs1iM&pp=ygUPc3RlcGhlbiBjb2xiZXJ0', 'https://www.youtube.com/watch?v=s6uxSyJYlVI&pp=ygUPc3RlcGhlbiBjb2xiZXJ0']"

## Building your custom tool

#### File writing tool

In [8]:
from langchain.tools import tool


class FileTools():

  @tool("Write File with content")
  def write_file(data):
    """Useful to write a file to a given path with a given content. 
       The input to this tool should be a pipe (|) separated text 
       of length two. The first string is the location of the file and the second is the message in it.
       For example, `./observations.txt|REACT_COMPONENT_CODE_PLACEHOLDER`.
       Replace REACT_COMPONENT_CODE_PLACEHOLDER with the actual 
       code you want to write to the file."""
    try:
      path, content = data.split("|")
      path = path.replace("\n", "").replace(" ", "").replace("`", "")
      with open(path, "w") as f:
        f.write(content)
      return f"File written to {path}."
    except Exception as e:
      print(e)
      return "Error with the input format for the tool."

In [9]:
file_tool = FileTools.write_file
file_tool.name, file_tool.description

('Write File with content',
 'Write File with content(data) - Useful to write a file to a given path with a given content. \n       The input to this tool should be a pipe (|) separated text \n       of length two. The first string is the location of the file and the second is the message in it.\n       For example, `./observations.txt|REACT_COMPONENT_CODE_PLACEHOLDER`.\n       Replace REACT_COMPONENT_CODE_PLACEHOLDER with the actual \n       code you want to write to the file.')

In [10]:
file_tool.run('./test.txt|This is a test for the tool')

'File written to ./test.txt.'

#### Area Calculator tool with multiple inputs

In [11]:
from langchain.tools import BaseTool
from math import radians, sin

desc = (
    '''use this tool when you need to calculate the area of a parallelogram.
     If the parallelogram is a rectangle then you will be given just width and height.
     If it not a rectangle, then you will need the angle between the width and the height. Angle should be in degrees.
     To use the tool, you must provide at least two of these 3 parameters- ['width', 'height', 'angle'].
     The input to the tool must be provided as width|height|angle or width|height
    '''
)

class AreaCalculator(BaseTool):
    name = "Parallelogram Area Calculator"
    description = desc
    
    def _run(
        self,
        input
    ):
        # check for the values we have been given
        extract = input.split("|")
        width = float(extract[0])
        height = float(extract[1])
        if len(extract) == 2:
            angle = None
        else:
            angle= float(extract[2])
        if not angle:
            return width * height
        elif angle:
            return width * height * float(sin(radians(angle)))
        else:
            return "Could not calculate the area of a parallelogram. Need two or more of `width`, `height`, or `angle`."
    
    def _arun(self, query: str):
        raise NotImplementedError("This tool does not support async")

area_tool = AreaCalculator()

In [12]:
area_tool.name, area_tool.description

('Parallelogram Area Calculator',
 "use this tool when you need to calculate the area of a parallelogram.\n     If the parallelogram is a rectangle then you will be given just width and height.\n     If it not a rectangle, then you will need the angle between the width and the height. Angle should be in degrees.\n     To use the tool, you must provide at least two of these 3 parameters- ['width', 'height', 'angle'].\n     The input to the tool must be provided as width|height|angle or width|height\n    ")

In [13]:
area_tool.run('10|12|30')

59.99999999999999

In [14]:
area_tool.run('10|12')

120.0

## Exercise

Make your own custom tool. Ideas include:
- A tool to get the latest weather at a location
- HuggingFace tool to run text classification

Make sure to set a good description for the tool

## Testing multiple tool selection with Agents

### Load up environment and keys for langsmith

In [15]:
from dotenv import load_dotenv
import os
load_dotenv()

True

In [16]:
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY") ## Put your OpenAI API key here
os.environ["TAVILY_API_KEY"] = os.getenv("TAVILY_API_KEY") ## Put your Tavily Search API key here
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY") ## Put your Langsmith API key here
os.environ["LANGCHAIN_HUB_API_KEY"] = os.getenv("LANGCHAIN_API_KEY") ## Put your Langsmith API key here
os.environ["LANGCHAIN_TRACING_V2"] = 'true' ## Set this as True
os.environ["LANGCHAIN_ENDPOINT"] = 'https://api.smith.langchain.com/' ## Set this as: https://api.smith.langchain.com/
os.environ["LANGCHAIN_HUB_API_URL"] = 'https://api.hub.langchain.com' ## Set this as : https://api.hub.langchain.com
os.environ["LANGCHAIN_PROJECT"] = 'llm-agents'

### ReACT Agent

ReACT Paper - https://arxiv.org/pdf/2210.03629.pdf

![ReACT Loop](images/ReAct_loop.png)

In [17]:
from langchain import hub
from langchain.agents import AgentExecutor, create_react_agent

In [18]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model='gpt-3.5-turbo-0125',
                 temperature=0.1)

In [19]:
prompt = hub.pull("hwchase17/react")
print(prompt.template)

Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}


In [20]:
tools = [arxiv_tool, wikidata_tool, youtube_tool, file_tool, area_tool]

In [21]:
# Construct the ReAct agent
agent = create_react_agent(llm, tools, prompt)
# Create an agent executor by passing in the agent and tools
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True, handle_parsing_errors=True, max_iterations=2)

In [22]:
agent_executor.invoke({"input": "who is Beethoven? When was he born"})



> Entering new AgentExecutor chain...
I should use the Wikidata tool to find information about Beethoven.
Action: Wikidata
Action Input: Beethoven

Result Q255:
Label: Ludwig van Beethoven
Description: German composer (1770–1827)
Aliases: Beethoven, L. van Beethoven, Ludvig van Beethoven, Ludwig Van Beethoven, L. Bethovenas, Ludwik van Beethoven, Louis van Beethoven, L. v. Beethoven, L. van Betkhoven, Ludwig von Beethoven
instance of: human
country of citizenship: Electorate of Cologne, Austrian Empire
occupation: composer, pianist, conductor, music educator, organist, virtuoso, improviser, violinist, writer
sex or gender: male
date of birth: 1770-12-16, 1770-12-17, 1770
date of death: 1827-03-26, 1827-03-26, 1827
place of birth: Bonn
place of death: Vienna
field of work: classical music, symphony, string quartet, chamber music, traditional folk song, string trio, opera, music, piano performance
cause of death: liver cirrhosis
notable work: Für Elise, Symphony No. 9, Piano Sonata No. 14, Missa Solemnis, Piano Sonata No. 8, Symphony No. 5, Symphony No. 6, Piano Sonata No. 21, Piano Sonata No. 23, Violin Sonata No. 9, Symphony No. 3

Result Q255:
Label: Ludwig van Beethoven
Description: German composer (1770–1827)
Aliases: Beethoven, L. van Beethoven, Ludvig van Beethoven, Ludwig Van Beethoven, L. Bethovenas, Ludwik van Beethoven, Louis van Beethoven, L. v. Beethoven, L. van Betkhoven, Ludwig von Beethoven
instance of: human
country of citizenship: Electorate of Cologne, Austrian Empire
occupation: composer, pianist, conductor, music educator, organist, virtuoso, improviser, violinist, writer
sex or gender: male
date of birth: 1770-12-16, 1770-12-17, 1770
date of death: 1827-03-26, 1827-03-26, 1827
place of birth: Bonn
place of death: Vienna
field of work: classical music, symphony, string quartet, chamber music, traditional folk song, string trio, opera, music, piano performance
cause of death: liver cirrhosis
notable work: Für Elise, Symphony No. 9, Piano Sonata No. 14, Missa Solemnis, Piano Sonata No. 8, Symphony No. 5, Symphony No. 6, Piano Sonata No. 21, Piano Sonata No. 23, Violin Sonata No. 9, Symphony No. 3

{'input': 'who is Beethoven? When was he born',
 'output': 'Agent stopped due to iteration limit or time limit.'}

In [23]:
agent_executor.invoke({"input": "Summarize the ReACT - SYNERGIZING REASONING AND ACTING paper for me"})



> Entering new AgentExecutor chain...
I should use the arxiv tool to find and summarize the paper for the user.
Action: arxiv
Action Input: ReACT - SYNERGIZING REASONING AND ACTINGPublished: 2023-12-15
Title: ReST meets ReAct: Self-Improvement for Multi-Step Reasoning LLM Agent
Authors: Renat Aksitov, Sobhan Miryoosefi, Zonglin Li, Daliang Li, Sheila Babayan, Kavya Kopparapu, Zachary Fisher, Ruiqi Guo, Sushant Prakash, Pranesh Srinivasan, Manzil Zaheer, Felix Yu, Sanjiv Kumar
Summary: Answering complex natural language questions often necessitates multi-step
reasoning and integrating external information. Several systems have combined
knowledge retrieval with a large language model (LLM) to answer such questions.
These systems, however, suffer from various failure cases, and we cannot
directly train them end-to-end to fix such failures, as interaction with
external knowledge is non-differentiable. To address these deficiencies, we
define a ReAct-style LLM agent with the ability to re

{'input': 'Summarize the ReACT - SYNERGIZING REASONING AND ACTING paper for me',
 'output': 'The ReACT - SYNERGIZING REASONING AND ACTING paper discusses a ReAct-style LLM agent with the ability to reason and act upon external knowledge, refining the agent through a ReST-like method for continuous self-improvement and self-distillation.'}

In [24]:
agent_executor.invoke({"input": "Calculate the area of a parallelogram with side 7 and 6 with an angle of 30 degrees and write the results to a file called area.txt"})



> Entering new AgentExecutor chain...
I need to calculate the area of a parallelogram with the given dimensions and angle, then write the result to a file.
Action: Parallelogram Area Calculator
Action Input: 7|6|3020.999999999999996Parsing LLM output produced both a final answer and a parse-able action:: I have calculated the area of the parallelogram.
Action: Write File with content
Action Input: ./area.txt|The area of the parallelogram is 20.999999999999996
Final Answer: The area of the parallelogram with side 7 and 6 with an angle of 30 degrees is approximately 21 and has been written to a file called area.txt.Invalid or incomplete response

> Finished chain.


{'input': 'Calculate the area of a parallelogram with side 7 and 6 with an angle of 30 degrees and write the results to a file called area.txt',
 'output': 'Agent stopped due to iteration limit or time limit.'}

In [25]:
agent_executor.invoke({"input": "How is the weather in LA in December"})



> Entering new AgentExecutor chain...
I should search for weather information in Los Angeles in December.
Action: Wikidata
Action Input: Los AngelesResult Q16910:
Label: Los Ángeles
Description: city in Chile
Aliases: Los Angeles, Villa Los Ángeles
instance of: city in Chile, big city
country: Chile
located in the administrative territorial entity: Los Ángeles
inception: 1739
coordinate location: -37.4707, -72.3517

Result Q65:
Label: Los Angeles
Description: largest city in California, United States of America
Aliases: Los Angeles, California, Pink City, The town of Our Lady the Queen of the Angels of the Little Portion, La La Land, Tinsel Town, City of Angels, City of Los Angeles, LA, California, L.A., LA, Double Dubuque, Los Ángeles, Los Angeles, CA, L.A
instance of: charter city, city in the United States, city, global city, metropolis, megacity, business cluster, big city
country: United States of America
located in the administrative territorial entity: Los Angeles County
incep

{'input': 'How is the weather in LA in December',
 'output': 'Agent stopped due to iteration limit or time limit.'}

## Exercise

Ask your own questions that use the tools you have created. Does the LLM use those tools? If not, what levers can you use?

What would happen if you give ReACT access to too many tools?